In [28]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import gauss
from itertools import product
seed(1)

In [214]:
#operator definitionis
N = 2
si = qeye(2)
sx = 0.5*sigmax()
sy = 0.5*sigmay()
sz = 0.5*sigmaz()

In [215]:
#make list of tensor products
sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    op_list = []
    for m in range(N):
        op_list.append(si)

    op_list[n] = sx
    sx_list.append(tensor(op_list))

    op_list[n] = sy
    sy_list.append(tensor(op_list))

    op_list[n] = sz
    sz_list.append(tensor(op_list))

In [216]:
#Construct Hamiltonian
B = 0
A0 = 1
H = B*sz_list[0] 
for n in range(N-1):
    H += A0*sz_list[0]*sz_list[n+1] + A0*sx_list[0]*sx_list[n+1] + A0*sy_list[0]*sy_list[n+1]
print(H)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.5   0.  ]
 [ 0.    0.5  -0.25  0.  ]
 [ 0.    0.    0.    0.25]]


In [217]:
#Find Groundstate
H.groundstate()

(-0.7500000000000001,
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[ 0.        ]
  [ 0.70710678]
  [-0.70710678]
  [ 0.        ]])

In [246]:
#Make |S> basis
Sbasis = []

#Define spin up as 0 and spin down as 1, with the index the position of the particle
perm = product([0,1],repeat = N)
permList = []
for i in list(perm):
    permList.append(i)

spin = np.asarray(permList)
index = np.zeros(2**N, dtype=int)
for j in range(2**N):
    for i in range(N):
        index[j] += 2**(i)*spin[j][N-1-i]
    Sbasis.append(basis(2**N,index[j]))

#print(Sbasis)
#there is some complication here, this order gives order of tp basis

#change spin to -1, and 1
for i in range(2**N):
    for j in range(N):
        if spin[i][j] == 0:
            spin[i][j] = 1
        else:
            spin[i][j] = -1
print(spin) 

[[ 1  1]
 [ 1 -1]
 [-1  1]
 [-1 -1]]


In [224]:
#Define RBM Parameters
M = 1
alp = M/N

a = [1 for j in range(N)]
b = [1 for i in range(M)]
W = [[1 for j in range(N)] for i in range(M)]


In [229]:
#Function to give RBM wavefuntion
def RBM_ansatz(a, b, W, Sbasis, spin, N, M):
    expTerm = np.zeros(2**N)
    visBias = np.zeros(2**N)
    WSum = np.zeros((2**N,M))
    hidSum = np.zeros((2**N,M))
    hidProduct = np.zeros(2**N)

    for i in range(2**N):
        for j in range (N):
            visBias[i] += spin[i][j]*a[j]
            for m in range(M):
                WSum[i][m] += W[m][j]*spin[i][j]
    for i in range(2**N):
        for m in range(M):
            hidSum[i][m] = np.cosh(WSum[i][m] + b[m])*2
            hidProduct[i] = np.prod(hidSum[i])
    for i in range(2**N):
        expTerm[i] = np.exp(visBias[i])


    #Wavefunction Definition
    psiMValues = np.zeros(2**N)
    for i in range(2**N):
        psiMValues[i] = expTerm[i]*hidProduct[i]
    psiM = Qobj(psiMValues)
    return psiM

psiM = RBM_ansatz(a, b, W, Sbasis, spin, N, M)
print(psiM)
 

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[148.78103854]
 [  3.08616127]
 [  3.08616127]
 [  0.41766651]]


In [244]:
#Variation Energy Definition
def varEnergy(psiM, H):
    E = psiM
    return E
E = varEnergy(psiM, H)
print (E)

print(H)
H.istp

#something happening with dimensions of matricies 



Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[148.78103854]
 [  3.08616127]
 [  3.08616127]
 [  0.41766651]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.5   0.  ]
 [ 0.    0.5  -0.25  0.  ]
 [ 0.    0.    0.    0.25]]


False

In [109]:
#Gradient Descent 
def updateParameters(a, b, W, learning rate):
    
    
    return a, b, W


In [ ]:
#Ground State Wavefunction
RBM_ansatz(a, b, W, Sbasis, spin, N, M)